<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Summary_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this notebook we will load the summarier models and generate summary we will save them and later evaluate in another notebook

In [1]:
!git clone https://github.com/dmmiller612/lecture-summarizer.git
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)
!pip install bert-extractive-summarizer

Cloning into 'lecture-summarizer'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 259 (delta 0), reused 3 (delta 0), pack-reused 255
Receiving objects: 100% (259/259), 280.43 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (507/507), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 507 (delta 242), reused 476 (delta 229), pack-reused 0
Receiving objects: 100% (507/507), 29.39 MiB | 22.24 MiB/s, done.
Resolving deltas: 100% (242/242), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import os
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
all_files = os.listdir("/content/data/")   
contents = []
for file_name in tqdm(all_files):
  dir_name = "/content/data/" + file_name
  with open(dir_name) as f:
    content = f.read()
    contents.append(content)

df = pd.DataFrame()

df['content'] = contents
df.head()

100%|██████████| 13/13 [00:00<00:00, 5410.39it/s]


,content
0,Welcome back to Pattern Recognition.\nToday we...
1,Welcome back to pattern recognition.\nToday we...
2,dr karen bell is the chair of the certificatio...
3,Welcome back to Pattern Recognition!\nToday we...
4,"In 2013, Grahame Greive, the leader of the gl..."


In [4]:
import textwrap

def generate_summary(pipe, paragraph):
  arr = textwrap.wrap(paragraph, 4000)
  summary = ''
  for i in arr:
    summary += pipe( i ) [0]["summary_text"].replace(" .<n>", ".\n")
  return summary

## BART

In [5]:
from transformers import pipeline, set_seed

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [6]:
bart_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(pipe, df['content'][i] ) 
  bart_summaries.append( summary )

df['bart'] = bart_summaries

 31%|███       | 4/13 [07:03<16:29, 109.92s/it]Your min_length is set to 56, but you input_length is only 15. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
 77%|███████▋  | 10/13 [12:21<03:20, 66.68s/it]Your min_length is set to 56, but you input_length is only 8. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
100%|██████████| 13/13 [14:58<00:00, 69.14s/it]


In [7]:
df.head()

,content,bart
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ..."


In [8]:
%cd ..
df.to_csv('summaries_c1_bart.csv', index=False)

/content


## PEGASUS

In [9]:
from transformers import pipeline, set_seed

pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [10]:
pegasus_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(pipe, df['content'][i] ) 
  pegasus_summaries.append( summary )

df['pegasus'] = pegasus_summaries
df.head()

 31%|███       | 4/13 [15:00<36:15, 241.77s/it]Your min_length is set to 32, but you input_length is only 14. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 128, but you input_length is only 14. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
 77%|███████▋  | 10/13 [26:31<07:10, 143.48s/it]Your min_length is set to 32, but you input_length is only 7. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 128, but you input_length is only 7. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
100%|██████████| 13/13 [31:30<00:00, 145.43s/it]


,content,bart,pegasus
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...,Support vector machines are a powerful classif...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...,Learn how to introduce the kernel PCA.\nLook i...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ...",There are many strongly held points of view ab...


## BERT Extractive 

In [11]:
from summarizer import Summarizer

summarizer = Summarizer()

bert_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = summarizer(df['content'][i] ) 
  bert_summaries.append( summary )

df['bert_extractive'] = bert_summaries
df.head()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

100%|██████████| 13/13 [00:37<00:00,  2.89s/it]


,content,bart,pegasus,bert_extractive
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...,Support vector machines are a powerful classif...,Today we want to start talking about support v...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...,Learn how to introduce the kernel PCA.\nLook i...,Today we want to look into a couple of the adv...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ...",There are many strongly held points of view ab...,"In 2013, Grahame Greive, the leader of the glo..."


## DistilBert Base Uncased

In [12]:
summarizer = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True) 

dist_bert_summaries = [] 
for i in tqdm(range( len(df) ) ): 
  summary = summarizer(df['content'][i] ) 
  dist_bert_summaries.append( summary ) 

df['dist_bert_extractive'] = dist_bert_summaries 

df.head() 

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

100%|██████████| 13/13 [00:15<00:00,  1.20s/it]


,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...,Support vector machines are a powerful classif...,Today we want to start talking about support v...,Today we want to start talking about support v...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...,Learn how to introduce the kernel PCA.\nLook i...,Today we want to look into a couple of the adv...,Today we want to look into a couple of the adv...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,,
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...,Today we want to look a bit more into convex o...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ...",There are many strongly held points of view ab...,"In 2013, Grahame Greive, the leader of the glo...","In 2013, Grahame Greive, the leader of the glo..."


In [14]:
df.to_csv('summaries.csv', index=False)

In [15]:
df.head()

,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...,Support vector machines are a powerful classif...,Today we want to start talking about support v...,Today we want to start talking about support v...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...,Learn how to introduce the kernel PCA.\nLook i...,Today we want to look into a couple of the adv...,Today we want to look into a couple of the adv...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,,
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...,Today we want to look a bit more into convex o...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ...",There are many strongly held points of view ab...,"In 2013, Grahame Greive, the leader of the glo...","In 2013, Grahame Greive, the leader of the glo..."


In [17]:
df.drop([2], inplace=True)

In [19]:
df.to_csv('summaries.csv', index=False)